# Explore the datacube concept and associated tools

In [ ]:
import datetime as dt
from pathlib import Path

from chaosmagpy.plot_utils import nio_colormap
import intake
import datashader
import holoviews
import hvplot
import hvplot.xarray
import numpy as np
import pandas as pd
import panel as pn
import pooch
import pyvista
import xarray as xr

pn.extension()
pyvista.set_jupyter_backend("trame")

## Download data to use locally

[Pooch](https://www.fatiando.org/pooch/) is a great library for downloading data. Here we use Pooch to download a zip file, validating the download using the hash, and unzipping the data ready for use.

In [ ]:
# Describe data name and locations
data_nc = {
    "path": Path("data"),
    "fname": "SwA_20140501-20190501_proc1.nc",
    "url": "https://drive.google.com/uc?export=download&confirm=no_antivirus&id=1qX-j_QWx0OQTh1HLUlHHcWdNhFOsczyM",
    "known_hash": "1b7a8cbc0cb1657f8d4444ae7f6bbab91841318e1a172fa1f8a487b9d9492912",
}

data_zarr = {
    "path": Path("data"),
    "fname": "datacube_test_SwA.zarr",
    "url": "https://drive.google.com/uc?export=download&confirm=no_antivirus&id=1a9ZhNuzBqCAjMzuwGG3MYw7MbJxNjMB_",
    "known_hash": "ab10b795320820d076f8741fad0aff415e8bd1c59997225c20ce69a25d3e3bdf",
}


def download_data(data=data_zarr, needs_unzip=False):
    """Download the data if we don't already have it"""
    full_path = data["path"] / data["fname"]
    if full_path.exists():
        print(f"Already found file: {full_path}")
    else:
        pooch.retrieve(
            url=data["url"],
            known_hash=data["known_hash"],
            path=data["path"],
            fname=f'{data["fname"]}.zip' if needs_unzip else data["fname"],
            progressbar=True,
            processor=pooch.Unzip(extract_dir=data["fname"]) if needs_unzip else None,
        )
    return full_path


data_path = download_data(data=data_nc, needs_unzip=False)
data_path = download_data(data=data_zarr, needs_unzip=True)

## Open data using intake

We could open the dataset using xarray directly like:

```python
ds = xr.open_dataset(data_path, engine="zarr", mode="r")
```

In [ ]:
cat = intake.open_catalog('catalog.yml')
list(cat)

In [ ]:
cat.datacube_local_A.container

In [ ]:
ds = cat.datacube_local_A.read_chunked()
ds

### Simplify the dataset

Having all the different variables in there at once makes it a bit noisy. Let's just look at a subset of the dataset:

In [ ]:
# Select particular data variables to be explored
ds = ds[
    [
        "B_NEC", "B_NEC_res_CHAOS-full",
        "Latitude", "Longitude", "MLT", "QDLat"
    ]
]
# Remove the metadata to reduce the clutter
ds.attrs = {}
ds

This dataset contains data/measurements:  
`"B_NEC"`, the magnetic vector, $B_{NEC}^{measured}$ in the NEC frame (North, East, Center)    
and data-model residuals:  
`"B_NEC_res_CHAOS-full"` = $B_{NEC}^{measured}$ - $B_{NEC}^{modelled}$, using the [CHAOS magnetic field model](http://www.spacecenter.dk/files/magnetic-models/CHAOS-7/)

These data are downsampled to 10-second resolution from the [original 1Hz dataset](https://swarmhandbook.earth.esa.int/catalogue/SW_MAGx_LR_1B) which themselves are derived from [50Hz measurements](https://swarmhandbook.earth.esa.int/catalogue/SW_MAGx_HR_1B). A sample between May 2014 and May 2019 has been prepared.


## Visualise data with hvPlot

[hvPlot](https://hvplot.holoviz.org/) is part of the [HoloViz](https://holoviz.org) ecosystem. The role of hvPlot is to "quickly return interactive HoloViews, GeoViews, or Panel objects from Pandas, Xarray, or other data structures". HoloViz is quite sprawling but the aim is to provide a layered approach to data analysis and visualisation:

<img src="https://holoviz.org/assets/shortcuts.png" alt="holoviz-shorcuts" style="width:50%;"/>

---

<img src="https://hvplot.holoviz.org/assets/diagram.svg" alt="hvplot-diagram" style="width:50%;"/>

### Plot a time series

The following will plot the time series of `B_NEC`...

- Directly plotting the full dataset won't work well, so we can slice out one day using `.sel`
- ... then we use the `.hvplot` accessor (this gets registered when we `import hvplot.xarray`)
- ... and we select a line plot and specify what is on the x and y axes

hvPlot automatically does different things depending on the dimensionality of the dataset. Here it adds a widget to select between the different dimensions of "Spacecraft" and "NEC".

In [ ]:
(
    ds
    .sel(Timestamp=slice("2015-01-01", "2015-01-02"))
    .hvplot.line(x="Timestamp", y="B_NEC")
)

In [ ]:
(
    ds
    .sel(Timestamp=slice("2015-01-01", "2015-01-02"))
    .hvplot.line(x="Timestamp", y="B_NEC_res_CHAOS-full")
)

### Downsample the data

For the next plots, let's simplify by downsampling the data so we can work with data all in memory:

In [ ]:
# Dataset downsampled by 1/30 (i.e. 5-minute sampling)
_ds = ds.isel(Timestamp=slice(0, -1, 30))
# Remove the unused spacecraft dimension for simplicity
_ds = _ds.squeeze("Spacecraft").drop("Spacecraft")

In [ ]:
_ds.load()

### Spatial plots

Let's look at the spatial variation. In these examples we will remove the time coordinates using `.drop` so that hvPlot does not do anything with them. Each figure shows how the three vector components ($B_C$, $B_E$, $B_N$) vary across the Earth.

In [ ]:
print("B_NEC: magnetic field measurements")
(
    _ds
    .drop("Timestamp")  # Squash out time so we only look at spatial variation
    .hvplot.scatter(
        x="Longitude", y="Latitude", c="B_NEC",  # Plot the geographic variation of magnetic field, B
        by="NEC", subplots=True,  # Make three plots, one for each vector direction
        rasterize=True,  # Uses datashader to adaptively sample the dataset based on the current view
        colorbar=True,
        hover=True,
        width=300, height=200,
        clim=(-50000, 50000), cmap=nio_colormap()
    )
)

In [ ]:
print("B_NEC_res_CHAOS-full: The effect of removing the full CHAOS model, comprising core, magnetosphere, and lithosphere. i.e. mostly space weather signals remaining")
(
    _ds
    .drop("Timestamp")
    .hvplot.scatter(
        x="Longitude", y="Latitude", c="B_NEC_res_CHAOS-full",  # Plot the magnetic residual (perturbation) instead
        by="NEC", subplots=True,
        rasterize=True,
        colorbar=True,
        hover=True,
        width=300, height=200,
        clim=(-50, 50), cmap=nio_colormap()  # Adjust scale down to +-50 nT
    )
)

In [ ]:
print("As above, but in QDLat / MLT coordinates (magnetic latitude and local time)")
(
    _ds
    .drop("Timestamp")
    .hvplot.scatter(
        x="MLT", y="QDLat", c="B_NEC_res_CHAOS-full",  # Plot against magnetic / local time coordinates
        by="NEC", subplots=True,
        rasterize=True,
        colorbar=True,
        hover=True,
        width=300, height=200,
        clim=(-50, 50), cmap=nio_colormap()
    )
)

This basically shows the part of the field which is unmodelled by CHAOS (principally the ionospheric field). The two-cell pattern across mid-latitudes in the daytime comes from the Sq system, and the large disturbances around the poles from field-aligned currents and auroral electrojets.

### hvPlot Explorer

Using complex calls like above are only possible with familiarity with both the data and hvplot. The [Explorer tool](https://hvplot.holoviz.org/user_guide/Explorer.html) aims to bridge this gap. Currently it only works with dataframes, so we have to first make the conversion.

It shows promise but it's too easy to make selections that are too heavy to compute/display. Crucially, data must be arranged in a certain for these automatic visualisations to work / make sense.

In [ ]:
# (we could use .to_dataframe() but let's simplify the data for this case)
_df = pd.DataFrame(
    data={
        "B_C": _ds["B_NEC"].sel(NEC="C"),
        "Latitude": _ds["Latitude"],
        "Longitude": _ds["Longitude"],
    },
    index=_ds["Timestamp"]
)
_df.index.name = "Timestamp"

In [ ]:
explorer = hvplot.explorer(_df)
explorer

To make some plots that are more meaningful, try selecting:

> `Fields/Kind`: scatter  
> `Fields/X`: Latitude  
> `Fields/y`: B_C

or

> `Fields/Kind`: scatter  
> `Fields/X`: Longitude  
> `Fields/y`: Latitude  
> `Operations/Rasterize`  
> `Colormapping/Color`: B_C

Now you can extract code to reproduce the custom plot you have built:

In [ ]:
explorer.plot_code()

### 3D visualisation with PyVista

[PyVista](https://docs.pyvista.org/version/stable/index.html) is one of several choices for 3D visualisation. Let's use it to attach one of those spatial plots onto a globe. (Some shortcuts are taken here...)

First let's set up a data array that we will visualise. We'll go back to the full dataset, `ds`. (in this case, without trying interative plots like above, the larger data volume is less of a problem)

In [ ]:
_ds = ds.squeeze("Spacecraft").drop("Spacecraft").unify_chunks()

In [ ]:
# _da = _ds["B_NEC"].sel(NEC="C").drop("NEC")
_da = _ds["B_NEC_res_CHAOS-full"].sel(NEC="C").drop("NEC")
_da.name = "B_C"
_da = _da.assign_coords(
    {
        "Latitude": _ds["Latitude"],
        "Longitude": _ds["Longitude"],
        "MLT": _ds["MLT"],
        "QDLat": _ds["QDLat"],
    }
)
_da

Now we use [Datashader](https://datashader.org/) to generate a raster from the data. Datashader aggregates data at each pixel location using a given method (in this case, evaluating the mean).

In [ ]:
cvs = datashader.Canvas(plot_width=1000, plot_height=500)
# agg = cvs.points(_da, x="Longitude", y="Latitude", agg=datashader.mean("B_C"))
agg = cvs.points(_da, x="MLT", y="QDLat", agg=datashader.mean("B_C"))
img = datashader.transfer_functions.shade(agg, cmap=nio_colormap())
img

This is the same image as one of the plots above but is made more intense because more data is included and a smoother mean is found at each pixel location.

In [ ]:
# Convert the DataShader image to a NumPy array and separate the RGBA channels
img_data = np.array(img, dtype=np.uint32)
r = np.bitwise_and(img_data, 0x000000FF).astype(np.uint8)
g = np.right_shift(np.bitwise_and(img_data, 0x0000FF00), 8).astype(np.uint8)
b = np.right_shift(np.bitwise_and(img_data, 0x00FF0000), 16).astype(np.uint8)
a = np.right_shift(np.bitwise_and(img_data, 0xFF000000), 24).astype(np.uint8)
# Stack the RGB channels into a 3D array (ignoring the alpha channel)
texture_data = np.dstack((r, g, b))

Map the raster above onto a sphere and display it.

In [ ]:
# Create a PyVista mesh and apply the texture
mesh = pyvista.Sphere()
mesh.texture_map_to_sphere(inplace=True, prevent_seam=False)
mesh.textures["data_texture"] = pyvista.numpy_to_texture(texture_data)
# Visualize the mesh with the texture in PyVista
plotter = pyvista.Plotter()
plotter.add_mesh(mesh)#, texture="data_texture")
plotter.show()

(still some work to do here to fix that seam!)

## To the cloud!

To try next: using a zarr remotely from S3 (add it to the intake catalog)